In [1]:
import pickle
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from typing import Dict, Any

In [2]:
import os
os.getcwd()

'/home/jonasmmiguel/Documents/learning/poli/thesis/wind-stf/prototyping'

In [3]:
models = pickle.load(open('../data/06_models/model.pkl/2020-10-09T00.59.06.757Z/model.pkl', 'rb'))

# MISC

In [3]:
df_infer = capacity_factors_daily_2000to2015 = pd.read_hdf(
    path_or_buf='../data/05_model_input/df_infer.hdf', 
    key='df_infer'
)

df_infer.head(3)

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.219507,0.269458,0.240540,0.263019,0.593148,0.331149,0.555176,0.284140,0.357407,0.410291,...,0.300818,0.363388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-02,0.083655,0.063571,0.043303,0.005984,0.080731,0.007986,0.066120,0.083709,0.078740,0.108224,...,0.150699,0.133930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-03,0.246707,0.229298,0.337413,0.047907,0.279092,0.136004,0.237430,0.320095,0.460150,0.516133,...,0.348737,0.297686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_infer_scaled = capacity_factors_daily_2000to2015 = pd.read_hdf(
    path_or_buf='../data/05_model_input/df_infer_scaled.hdf', 
    key='df_infer_scaled'
)

df_infer_scaled.head(3)

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,6.030707,6.332357,6.507886,6.628375,6.878469,6.701568,6.856132,6.266206,6.590818,6.636133,...,6.701568,6.824102,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
2013-01-02,5.232691,5.199338,5.104712,4.391300,5.327520,4.303404,5.254946,5.191001,5.556967,5.489596,...,6.141913,5.918106,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08
2013-01-03,6.133282,6.227721,6.755229,5.601776,6.190702,6.046516,6.107795,6.370123,6.856132,6.913633,...,6.856132,6.555127,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08


In [18]:
splits_positions = pickle.load(open('../data/05_model_input/cv_splits_positions.pkl/2020-10-09T00.50.52.126Z/cv_splits_positions.pkl', 'rb'))
splits_positions

{'pass 1': {'train': slice(0, 542, None), 'val': slice(542, 549, None)},
 'pass 2': {'train': slice(0, 719, None), 'val': slice(719, 726, None)},
 'pass 3': {'train': slice(0, 896, None), 'val': slice(896, 903, None)}}

In [19]:
modeling = {
  'approach': 'HW-ES',
  'mode': 'districtwise',
  'model_inference_window': {
    'start': '2013-01-01',
    'end': '2015-06-22',
  },
  'test_window': {
    'start': '2015-06-23',
    'end': '2015-06-29',
  },
  'preprocessing': ['get_quantile_equivalent_normal_dist', 'make_strictly_positive'],
  'hyperpars': {
    'trend': 'additive',
      'seasonal': 'multiplicative',
      'seasonal_periods': 7,
  },
  'targets': ['DEF0C', 'DE111'],
}

In [20]:
class ForecastingModel:
    def __init__(self, df, modeling: Dict[str, Any]):
        self.modeling = modeling
        self.df = df
        self.targets = df.columns
        self.submodels_ = {}
        self.model_ = None

    def fit(self):

        if self.modeling['mode'] == 'districtwise':

            if self.modeling['approach'] == 'HW-ES':
                self.submodels_ = {
                    district: ExponentialSmoothing(self.df[district], **self.modeling['hyperpars'])
                    for district in self.targets
                }
                return self.submodels_

        elif self.modeling['mode'] == 'spatio-temporal':  # i.e. all districts at once

            if self.modeling['approach'] == 'RNN-ES':
                self.model_ = None

            elif self.modeling['approach'] == 'GWNet':
                self.model_ = None

            return self.model_

        else:
            return NotImplementedError('')



    def predict(self, start, end, transformer):
        y_hat = self.model_.predict(start, end)
        y_hat_unscaled = transformer.inverse_transform(y_hat)
        return y_hat_unscaled

In [22]:
df = df_infer_scaled

In [24]:
submodel = ExponentialSmoothing(df_infer_scaled['DEF0C'], **modeling['hyperpars']).fit()

In [25]:
df_infer_scaled.index[720]

Timestamp('2014-12-22 00:00:00', freq='D')

In [26]:
submodel.predict(
    start=df_infer_scaled.index[720],
    end=df_infer_scaled.index[721]
)

date
2014-12-22    6.855705
2014-12-23    7.077107
Freq: D, dtype: float64

In [28]:
submodel.predict(
    start = df_infer_scaled[ splits_positions['pass 3']['val'] ].index[0],
    end = df_infer_scaled[ splits_positions['pass 3']['val'] ].index[-1]
)

date
2015-06-16    4.833661
2015-06-17    4.843037
2015-06-18    5.229563
2015-06-19    5.582237
2015-06-20    5.748552
2015-06-21    5.722404
2015-06-22    4.947315
Freq: D, dtype: float64

In [31]:
targets = modeling['targets']

In [32]:
submodels = {
    district: ExponentialSmoothing(df_infer_scaled['DEF0C'], **modeling['hyperpars']).fit()
    for district in targets
}

In [35]:
yhat = {}
for district in targets:
    yhat[district] = submodels[district].predict(
        start = df_infer_scaled[ splits_positions['pass 3']['val'] ].index[0],
        end = df_infer_scaled[ splits_positions['pass 3']['val'] ].index[-1]
    )
    
yhat = pd.DataFrame(yhat)

In [36]:
yhat

,DEF0C,DE111
date,,
2015-06-16,4.833661,4.833661
2015-06-17,4.843037,4.843037
2015-06-18,5.229563,5.229563
2015-06-19,5.582237,5.582237
2015-06-20,5.748552,5.748552
2015-06-21,5.722404,5.722404
2015-06-22,4.947315,4.947315


In [8]:
import sys
sys.path.append('..')

In [9]:
scaler = pickle.load(open('../data/05_model_input/scaler.pkl/2020-10-09T02.14.26.104Z/scaler.pkl','rb'))

In [37]:
scaler

Pipeline(steps=[('quantiletransformer',
                 QuantileTransformer(output_distribution='normal',
                                     random_state=0)),
                ('makestrictlypositive', MakeStrictlyPositive())])

In [42]:
df_preds = pd.DataFrame(
    data=None,
    columns=df_infer_scaled.columns,
    index=yhat.index,
)

df_preds.update(yhat)

df_preds.head()

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2015-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df_preds['DEF0C']

date
2015-06-16    4.83366
2015-06-17    4.84304
2015-06-18    5.22956
2015-06-19    5.58224
2015-06-20    5.74855
2015-06-21     5.7224
2015-06-22    4.94731
Freq: D, Name: DEF0C, dtype: object

In [55]:
df_preds_unscaled = pd.DataFrame(
    data = scaler.inverse_transform(df_preds),
    columns = df_preds.columns,
    index = df_preds.index,
)

In [56]:
df_preds_unscaled['DEF0C']

date
2015-06-16    0.130742
2015-06-17    0.132158
2015-06-18    0.207454
2015-06-19    0.294201
2015-06-20    0.329595
2015-06-21    0.324806
2015-06-22    0.147588
Freq: D, Name: DEF0C, dtype: float64